# Más sobre objetos

Hemos visto que si bien en Python todo es un objeto, esto es, una instancia de alguna clase con sus atributos y métodos, los mismos son públicos. Esto otorga gran versatilidad a la programación, pero puede ser un inconveniente a la hora de mantener código, debido a que el pretendido encapsulamiento de datos y funciones en una clase es pura responsabilidad del programador. Aquí veremos algunas facilidades que otorga Python para ayudar a crear clases robustas.


## El decorador `@classmethod`

En nuestra versión de la clase `Punto`, teníamos la capacidad de ir registrando el número de puntos que se van inicializando,

In [ ]:
class Punto:
  "Clase para describir un punto en el espacio"

  num_puntos = 0

  def __init__(self, x=0, y=0, z=0):
    "Inicializa un punto en el espacio"
    self.x = x
    self.y = y
    self.z = z
    Punto.num_puntos += 1
    return None

  def __del__(self):
    "Borra el punto y actualiza el contador"
    Punto.num_puntos -= 1

In [ ]:
p1 = Punto(1,1,1)
p2 = Punto()
print('Número de puntos:', Punto.num_puntos)
del p2
print('Número de puntos:', Punto.num_puntos)

Claramente la variable `num_puntos` es un dato de la clase `Punto`, y no de una instancia particular de la misma. Para mejorar la organización de este tipo de datos o métodos asociados a una clase, Python provee el decorador `@classmethod`: 

In [ ]:
class Punto:
  "Clase para describir un punto en el espacio"

  num_puntos = 0
  
  def __init__(self, x=0, y=0, z=0):
    "Inicializa un punto en el espacio"
    self.x = x
    self.y = y
    self.z = z
    Punto.num_puntos += 1
    return None

  def __del__(self):
    "Borra el punto y actualiza el contador"
    Punto.num_puntos -= 1

  @classmethod
  def total(cls):
    "Imprime el número total de puntos"
    print(f"En total hay {cls.num_puntos} puntos definidos")


Así como `self` debe ser el primer argumento de los métodos de instancia, `cls` se refiere a la propia clase y debe ser el primer argumento del método de clase decorado por `@classmethod`. De la misma forma, se utiliza la palabra `cls` por convención, podría ser cualquier otra siempre que se mantenga la consistencia interna.

In [ ]:
p1 = Punto(1,1,1)
p2 = Punto()
Punto.total()
del p2
Punto.total()

Otro ejemplo interesante es crear constructores alternativos:

In [ ]:
class Persona:
    def __init__(self, nombre, edad):
        self.nombre = nombre
        self.edad = edad

    @classmethod
    def desde_cadena(cls, cadena):
        nombre, edad = cadena.split(", ")
        return cls(nombre, int(edad))  # Devuelve una nueva instancia

# Crear una instancia desde una cadena
p3 = Persona.desde_cadena("Lionel, 37")


In [ ]:
print(p3.nombre)  
print(p3.edad)    

> En el ejercicio de `Polinomio`, puede transformar la función `from_string` requerida a un método de clase, de forma tal que se pueda crear un polinomio como:
```python
p1 = Polinomio.from_string("4 x^3 + 3 x^2 + 2.1 x + 1")

## _Getters_  y _Setters_ 



### Función `property` 

Volvamos a nuestra clase `Punto` y veamos cómo podemos mejorarla para no incurrir en posibles errores.

In [ ]:
class Punto:
  "Clase para describir un punto en el espacio"

  num_puntos = 0
  
  def __init__(self, x=0, y=0, z=0):
    "Inicializa un punto en el espacio"
    self.x = x
    self.y = y
    self.z = z
    Punto.num_puntos += 1
    return None

  def __del__(self):
    "Borra el punto y actualiza el contador"
    Punto.num_puntos -= 1

  def __str__(self):
    return f"Punto en el espacio con coordenadas: x = {self.x}, y = {self.y}, z = {self.z}"

  def __repr__(self):
    return f"Punto(x = {self.x}, y = {self.y}, z = {self.z})"

  def __call__(self):
    return "Ejecuté el objeto: {}".format(self)
#    return str(self)
#    return "{}".format(self)

  @classmethod
  def total(cls):
    "Imprime el número total de puntos"
    print(f"En total hay {cls.num_puntos} puntos definidos")
    

In [ ]:
P1 = Punto('a',1,2.) 
print(P1)
P1

Esto ocurrió porque nos olvidamos de verificar que los argumentos son del tipo correcto. Una manera de solucionarlo es chequear que los valores son del tipo correcto al crear el objeto, como hicimos anteriormente:

In [ ]:
class Punto:
  "Clase para describir un punto en el espacio"

  num_puntos = 0
  
  def __init__(self, x=0, y=0, z=0):
    "Inicializa un punto en el espacio"
    if not (isinstance(x, (int, float)) and isinstance(y, (int, float)) and isinstance(z, (int, float))):
      raise TypeError("x, y, z deben ser números enteros o flotantes")
    self.x = x
    self.y = y
    self.z = z
    Punto.num_puntos += 1
    

  def __del__(self):
    "Borra el punto y actualiza el contador"
    Punto.num_puntos -= 1

  def __str__(self):
    return f"Punto en el espacio con coordenadas: x = {self.x}, y = {self.y}, z = {self.z}"

  def __repr__(self):
    return f"Punto(x = {self.x}, y = {self.y}, z = {self.z})"

  @classmethod
  def total(cls):
    "Imprime el número total de puntos"
    print(f"En total hay {cls.num_puntos} puntos definidos")
    

In [ ]:
Punto('a',1,2.)

 Sin embargo aún tendremos problemas si los usuarios lo modifican luego de crearlo:

In [ ]:
P1 = Punto(1,2,3)

In [ ]:
P1.x = 'b'
P1

Una solución a esto es hacer las componentes "privadas" (por convención) para que los usuarios no la modifiquen directamente. El problema es que los usuarios tienen que poder acceder y modificarla. Una solución es crear métodos para darle valores (*setter*) y tomarlos (*getter*)

In [ ]:
class Punto:
  "Clase para describir un punto en el espacio"

  num_puntos = 0
  
  def __init__(self, x=0, y=0, z=0):
    "Inicializa un punto en el espacio"
    Punto.num_puntos += 1
    self.set_coordenadas(x,y,z)
    return None

  def get_coordenadas(self):
    return self._x, self._y, self._z

  def set_coordenadas(self, x=0, y=0, z=0):
    if not (isinstance(x, (int, float)) and isinstance(y, (int, float)) and isinstance(z, (int, float))):
      raise TypeError("x, y, z deben ser números enteros o flotantes")
    self._x = x
    self._y = y
    self._z = z
      
  def __del__(self):
    "Borra el punto y actualiza el contador"
    Punto.num_puntos -= 1

  def __str__(self):
    return f"Punto en el espacio con coordenadas: x = {self._x}, y = {self._y}, z = {self._z}"

  def __repr__(self):
    return f"Punto(x = {self._x}, y = {self._y}, z = {self._z})"

  @classmethod
  def total(cls):
    "Imprime el número total de puntos"
    print(f"En total hay {cls.num_puntos} puntos definidos")

> _Por convención_ se denota a las variables privadas con un guión bajo antes del nombre, ej, `_x`. 

In [ ]:
P1 = Punto(3,2,4.5)

In [ ]:
P2 = Punto(3,2,"hola")

In [ ]:
print(P1.get_coordenadas())

In [ ]:
# Tomemos el valor de la componente x
a = P1.x

In [ ]:
P1.__dict__

In [ ]:
# Se puede hacer, pero no queremos que se haga!
P1._x

Esto es un problema!
Cambiamos la clase haciendo las variables x,y,z 'privadas', pero eso implicó cambiarles el nombre a `_x,_y,_z`, si ya hay una versión de esta clase utilizada en otros programas y acceden a `x,y,z` ( es un comportamiento muy razonable querer modificar las coordenadas del punto...). Hicimos un cambio necesario, pero que puede afectar uno o más programas existentes y habría que rastrear y modificar todos (asumiendo que son nuestros). Para ello existe la función `property()` y el decorador correspondiente `@property`

In [ ]:
class Punto:
  "Clase para describir un punto en el espacio"

  num_puntos = 0
  
  def __init__(self, x=0, y=0, z=0):
    "Inicializa un punto en el espacio"
    Punto.num_puntos += 1
    self.set_coordenadas(x,y,z)
    return None

  def get_coordenadas(self):
    return self._x, self._y, self._z

  def get_x(self):
    return self._x
      
  def get_y(self):
    return self._y
      
  def get_z(self):
    return self._z

  def set_x(self, x):
    if not isinstance(x, (int, float)):
      raise TypeError("x debe ser número entero o flotante")        
    self._x = x
        
  def set_y(self, y):
    if not isinstance(y, (int, float)):
      raise TypeError("y debe ser número entero o flotante")        
    self._y = y
        
  def set_z(self, z):
    if not isinstance(z, (int, float)):
      raise TypeError("z debe ser número entero o flotante")        
    self._z = z
    
  def set_coordenadas(self, x=0, y=0, z=0):
    if not (isinstance(x, (int, float)) and isinstance(y, (int, float)) and isinstance(z, (int, float))):
      raise TypeError("x, y, z deben ser números enteros o flotantes")
    self._x = x
    self._y = y
    self._z = z
      
  def __del__(self):
    "Borra el punto y actualiza el contador"
    Punto.num_puntos -= 1

  def __str__(self):
    return f"Punto en el espacio con coordenadas: x = {self._x}, y = {self._y}, z = {self._z}"

  def __repr__(self):
    return f"Punto(x = {self._x}, y = {self._y}, z = {self._z})"

  def __call__(self):
    return "Ejecuté el objeto: {}".format(self)
#    return str(self)
#    return "{}".format(self)

  @classmethod
  def total(cls):
    "Imprime el número total de puntos"
    print(f"En total hay {cls.num_puntos} puntos definidos")

  x = property(get_x, set_x)
  y = property(get_y, set_y)
  z = property(get_z, set_z)
  

In [ ]:
P1 = Punto(2,4,6)

In [ ]:
a = P1.x

In [ ]:
print(a, P1.y)

In [ ]:
P1.x = 3

In [ ]:
P1

In [ ]:
P1.y = '5'

In [ ]:
P1.__dict__

## Más sobre herencia

Vimos cómo usar herencia para que una clase pueda derivarse desde otra, en nuestro caso, `Vector` era derivado de la clase `Punto`. Otro ejemplo sencillo podría ser el siguiente:

In [ ]:
class Persona:
    def __init__(self, nombre, edad):
        self.nombre = nombre
        self.edad = edad

    def presentarse(self):
        return f"Hola, soy {self.nombre} y tengo {self.edad} años."

# Clase derivada: Estudiante
class Estudiante(Persona):

    def __init__(self, nombre, edad, carrera):
        # Se llama al constructor de Persona directamente
        Persona.__init__(self, nombre, edad)
        self.carrera = carrera

    def presentarse(self):
        return f"Hola, soy {self.nombre}, tengo {self.edad} años y estudio {self.carrera}."


Aquí la clase `Estudiante` deriva de la clase `Persona`, y en el constructor (`__init__`) de la clase estudiante, utilizamos el constructor de la clase base `Persona`:

In [ ]:
pedro = Estudiante('Pedro', 25, 'Física')
pedro.presentarse()                

Otra posibilidad que brinda Python para referirse a los constructores de la clase base es utilizar la función `super()`:

In [ ]:
class Persona:
    def __init__(self, nombre, edad):
        self.nombre = nombre
        self.edad = edad

    def presentarse(self):
        return f"Hola, soy {self.nombre} y tengo {self.edad} años."

# Clase derivada: Estudiante
class Estudiante(Persona):
    def __init__(self, nombre, edad, carrera):
        # Llamamos al constructor de la clase base con super()
        super().__init__(nombre, edad)
        self.carrera = carrera  # Nuevo atributo para Estudiante

    def presentarse(self):
        # Reutilizamos el método de la clase base y agregamos más información
        return f"{super().presentarse()} Estoy estudiando {self.carrera}."


In [ ]:

# Uso de las clases
carlos = Persona("Carlos", 20)
maria = Estudiante("María", 20, "Ingeniería")

In [ ]:
print(carlos.presentarse())
print(maria.presentarse())

----

## Ejercicios 08 (a)

1. Cree una nueva clase `Materia` que describa una materia que se dicta en el IB. La clase debe contener información sobre el nombre de la materia, los alumnos que la cursan, y los docentes que la dictan. Utilice las clases `Persona` y `Estudiante` y, si es necesario, cree nuevas clases. 
Además debe proveer los siguientes métodos:
  - `agrega_estudiante` que agrega un estudiante al curso
  - `agrega_docente` que agrega un docente al curso
  - `imprime_estudiantes` que lista los estudiantes del curso






  
----
